# 7.1 데커레이터 기본 지식
- 함수를 인수로 받는 callable 객체
- 함수에 처리를 수행 후 반환 or 함수를 다른 callable 객체로 대체

In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [2]:
@deco
def target():
    print("running target()")
    
target() # 실제로는 inner()를 실행한다.

running inner()


In [3]:
target # inner()를 가리키고 있음

<function __main__.deco.<locals>.inner()>

# 7.2 데커레이터 실행 시점 
- 함수정의 직후 --> 파이썬이 모듈을 로딩하는 시점에 실행됨

In [6]:
!cat registration.py


registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1() # 이 경우 register()가 원래 함수를 반환하므로 f1()이 실행됨
    f2()
    f3()

if __name__=='__main__':
    main()

In [5]:
!python3 registration.py

running register(<function f1 at 0x103e67790>)
running register(<function f2 at 0x103e675e0>)
running main()
registry -> [<function f1 at 0x103e67790>, <function f2 at 0x103e675e0>]
running f1()
running f2()
running f3()


# 7.3 데커레이터로 개선한 전략 패턴 
- promos 리스트에 함수명을 반복하는 문제를 데커레이터로 해결

In [7]:

promos = [] # global 객체

def promotion(promo_func):
    """promotion() 데커레이터는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order): 
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)


# 이 방법은 다음과 같은 장점을 가진다.

# 프로모션 전략 함수명이 특별한 형태로 되어 있을 필요가 없다.
# @promotion 데커레이터는 임시로 어떤 프로모션을 배제할 수 있다.
# 프로모션 할인 전략을 구현한 함수는 @promotion 데커레이터가 적용되는 한 어느 모듈에서든 정의할 수 있다.

# 7.4 변수 범위 규칙 
- 함수의 매개변수와 정의되지 않은 변수 테스트 

In [9]:
b = 6 # 정의되지 않으면 오류
def f1(a):
    print(a)
    print(b)
    
f1(3)

3
6


In [10]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9 # 컴파일시, b를 지역변수로 판단하게 됨 

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [11]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
    
f3(3), b

3
6


(None, 9)

In [12]:
# dis 모듈을 사용하면 파이썬 함수를 쉽게 바이트코드로 디스어셈블할 수 있다.
from dis import dis
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [13]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 7.5 클로저 
- 함수 본체에서 정의되지 않았지만 참조하는 non-global 변수를 포함한 확장 범위를 가진 함수 

In [15]:
# class 를 정의하여 구현
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [16]:
# 고위 함수를 이용하여 구현
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

# 어디에서 series 변수를 찾는가?? 
# series 는 자유변수로 지역 범위에 바인딩 되어있지 않다 

In [17]:
avg = make_averager()
avg(10)
avg(11)
avg(12)

11.0

In [20]:
avg.__code__.co_varnames, avg.__code__.co_freevars
# series에 대한 바인딩은 avg() 함수의 __closure__ 속성에 저장

(('new_value', 'total'), ('series',))

In [21]:
avg.__closure__, avg.__closure__[0].cell_contents

((<cell at 0x1098305b0: list object at 0x10447cb00>,), [10, 11, 12])

In [ ]:
# 클로저는 함수를 정의할 때 존재하던 자유 변수에 대한 바인딩을 유지하는 함수다. 
# 따라서 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.
# 함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우 뿐이다.

# 7.6 nonlocal 선언
- 매번 합계를 다시 계산하는 것은 비효율적이다. 

In [22]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

In [23]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [24]:
# series는 가변형 리스트였으므로 문제가 없었다. 
# 그러나 count = count + 1과 같은 문장으로 변수를 다시 바인딩하면 암묵적으로 count라는 지역 변수를 만든다. 
# count가 더이상 자유 변수가 아니므로 클로저에 저장되지 않는다.

In [25]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total # nonlocal 선언을 통해 자유변수로 만듬
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

10.0

# 7.7 데커레이터 구현
- 함수 호출시, 시간을 측정하는 예시

In [26]:
! cat clockdeco.py

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [27]:
import time
from clockdeco import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*'*40, 'Calling snooze(.123)')
snooze(.123)
print('*'*40, 'Calling factorial(6)')
print('6! =', factorial(6))
# clock()은 factorial() 함수를 func 인수로 받아 clocked() 함수를 factorial에 할당한다.

**************************************** Calling snooze(.123)
[0.12491139s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000075s] factorial(1) -> 1
[0.00004223s] factorial(2) -> 2
[0.00006319s] factorial(3) -> 6
[0.00007964s] factorial(4) -> 24
[0.00012091s] factorial(5) -> 120
[0.00014683s] factorial(6) -> 720
6! = 720


In [29]:
factorial.__name__ # factorial이 clocked를 참조하는 것을 확인

'clocked'

In [30]:
# clock() 데커레이터는 단점이 몇 개 있다. 
# 키워드 인수를 지원하지 않으며, 
# 데커레이트된 함수의 __name__과 __doc__ 속성을 가린다. 
# 
# 이를 functools.wraps() 데커레이터를 이용해서 해결

In [31]:
!cat clockdeco2.py


import time
import functools

def clock(func):
    @functools.wraps(func) # 데커레이터 적용
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [32]:
import time
from clockdeco2 import clock

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

print('*'*40, 'Calling snooze(.123)')
snooze(.123)
print('*'*40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12353396s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000405s] factorial(1) -> 1
[0.00004315s] factorial(2) -> 2
[0.00006509s] factorial(3) -> 6
[0.00008488s] factorial(4) -> 24
[0.00010324s] factorial(5) -> 120
[0.00012708s] factorial(6) -> 720
6! = 720


In [33]:
factorial.__name__ # 원래 이름으로 표시~

'factorial'

# 7.8 표준 라이브러리에서 제공하는 데커레이터
- 메서드 데커레이터 : property(), classmethod(), staticmethod()
-  lru_cache()와 singledispatch()

## 7.8.1 functools.lru_cache() 메모이제이션

In [36]:

from clockdeco2 import clock

@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(6)) # 매번 계산을 다시해야되서 낭비가 심함 

[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00051308s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00053906s] fibonacci(3) -> 2
[0.00000072s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00002694s] fibonacci(2) -> 1
[0.00063992s] fibonacci(4) -> 3
[0.00000119s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00015402s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00023603s] fibonacci(3) -> 2
[0.00114894s] fibonacci(5) -> 5
[0.00000000s] fibonacci(1) -> 1
[0.00000119s] fibonacci(0) -> 0
[0.00064492s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00069690s] fibonacci(3) -> 2
[0.00000095s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00052094s] fibonacci(2) -> 1
[0.00132990s] fibonacci(4) -> 3
[0.00254893s] fibonacci(6) -> 8
8


In [37]:
import functools

@functools.lru_cache() # lru cache 로 개선
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00030422s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00033402s] fibonacci(4) -> 3
[0.00000095s] fibonacci(5) -> 5
[0.00036001s] fibonacci(6) -> 8
8


#### lru_cache args
- maxsize : 얼마나 많은 호출을 저장할지 결정
- typed : True 일 경우 arg 의 자료형이 다르면 결과를 따로 저장
- lru_cache가 적용되는 함수의 arg 는 모두 해시 가능해야한다.

# 7.8.2 단일 디스패치를 이용한 범용 함수 
- 웹어플리케이션을 디버깅하는 도구를 가정 
- str : 개행 문자를 <br>\n으로 대체하고 <pre> 대신 <p> 태그를 사용한다.
- int : 숫자를 10진수와 16진수로 보여준다.
- list : 각 항목을 자료형에 따라 포맷한 HTML 리스트를 출력한다.


In [38]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [39]:
# 일반 함수를 singledispatch()로 데커레이트하면 이 함수는 범용 함수(generic function)이 됨 
# 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.

from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str) # 각각의 특화된 함수는 @<기반_함수>.register(<객체형>)으로 데커레이트
def _(text): # 특화된 함수의 이름은 필요없음
    content = html.escape(text).replace('\n','<br>\n')
    return '<p>{}</p>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>{}</li>\n</ul>'.format(inner)

# int나 list와 같은 구상 클래스보다 numbers.Integral이나 abc.MutableSequence와 같은 추상 베이스 클래스를 처리하도록 특화된 함수를 등록하는 것이 좋다.

# 7.9 데커레이터 누적 
- 하나의 함수 f()에 두 데커레이터 @d1과 @d2를 차례대로 적용하면 결과는 f = d1(d2(f))와 같다.

# 7.10 매개변수화된 데커레이터
- 다른 인수를 받는 데커레이터

In [42]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x109ce4940>)
running main()
registry -> [<function f1 at 0x109ce4940>]
running f1()


In [43]:
# register()가 등록하는 함수를 활성화 혹은 비활성화하기 쉽게 만들기 위해 선택 인수 active를 도입하자.


registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        if active:
            registry.add(func)
        else:
            registry.discard(func)
            
        return func
    return decorate

@register(active=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')

running register(active=False)->decorate(<function f1 at 0x109ce43a0>)
running register(active=True)->decorate(<function f2 at 0x109ce4280>)


# 7.10.2 매개변수화된 clock 데커레이터

In [44]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT): # 매개변수화된 데커레이터 팩토리
    def decorate(func): # 실제 데커레이터
        def clocked(*_args): #데커레이트된 함수를 래핑
            t0 = time.time()
            _result = func(*_args) # 함수의 실제 연산 결과
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result) # _result를 문자화
            print(fmt.format(**locals())) # fmt가 clocked()의 지역 변수를 모두 참조
            return _result # clocked는 데커레이트된 함수를 대체하므로, 원래 함수가 반환하는 값을 반환
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12455297s] snooze(0.123) -> None
[0.12724876s] snooze(0.123) -> None
[0.12382317s] snooze(0.123) -> None


In [46]:
import time

@clock('{name}: {elapsed}s') # 포멧을 인자로 전달
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12315106391906738s
snooze: 0.12319207191467285s
snooze: 0.12584900856018066s


In [47]:
import time

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.127s
snooze(0.123) dt=0.125s
snooze(0.123) dt=0.126s
